# Term deposit marketing

## Plan

## Load dataset

In [32]:
import pathlib
import os
import toml
import pandas as pd

import pycaret.classification as pycaret
from IPython.display import display, Markdown
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix

# Add sys to path
import sys

sys.path.append("./term_deposit")
import term_deposit as td

# Change the working directory to the root of the project
while os.path.basename(cwd := os.getcwd()) != "2-term_deposit_marketing":
    print(cwd)
    os.chdir("..")

# Load the configuration file
config = toml.load('config.toml')
target = 'y'

In [33]:
# Load the dataset
df = pd.read_parquet(config['paths']["data"]['post'])

# Display basic information about the dataset
df.info(), display(df.sample(3)), display(df.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 36 columns):
 #   Column               Non-Null Count  Dtype
---  ------               --------------  -----
 0   number_calls         40000 non-null  int64
 1   y                    40000 non-null  int64
 2   job_blue-collar      40000 non-null  uint8
 3   job_entrepreneur     40000 non-null  uint8
 4   job_housemaid        40000 non-null  uint8
 5   job_management       40000 non-null  uint8
 6   job_retired          40000 non-null  uint8
 7   job_self-employed    40000 non-null  uint8
 8   job_services         40000 non-null  uint8
 9   job_student          40000 non-null  uint8
 10  job_technician       40000 non-null  uint8
 11  job_unemployed       40000 non-null  uint8
 12  marital_married      40000 non-null  uint8
 13  marital_single       40000 non-null  uint8
 14  education_secondary  40000 non-null  uint8
 15  education_tertiary   40000 non-null  uint8
 16  default_yes          4

,number_calls,y,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,marital_married,marital_single,education_secondary,education_tertiary,default_yes,housing_yes,loan_yes,contact_telephone,month_aug,month_dec,month_feb,month_jan,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,high_balance,duration_q1,duration_q2,duration_q3,age_group_18-30,age_group_60+
32566,1,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
28547,2,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0
6456,2,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0


,number_calls,y,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,marital_married,marital_single,education_secondary,education_tertiary,default_yes,housing_yes,loan_yes,contact_telephone,month_aug,month_dec,month_feb,month_jan,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,high_balance,duration_q1,duration_q2,duration_q3,age_group_18-30,age_group_60+
count,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.00000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000
mean,2.882175,0.072400,0.234575,0.035125,0.027175,0.204150,0.035925,0.035350,0.09775,0.013100,0.171300,0.027600,0.609650,0.272225,0.524825,0.280150,0.020225,0.600775,0.173250,0.058025,0.130375,0.000325,0.057400,0.029400,0.159500,0.118350,0.006450,0.338300,0.089950,0.002000,0.397950,0.335675,0.331925,0.332400,0.145975,0.005850
std,3.239051,0.259152,0.423738,0.184098,0.162595,0.403084,0.186106,0.184665,0.29698,0.113704,0.376776,0.163826,0.487835,0.445111,0.499390,0.449078,0.140771,0.489745,0.378468,0.233794,0.336720,0.018025,0.232608,0.168927,0.366146,0.323026,0.080053,0.473137,0.286114,0.044677,0.489481,0.472232,0.470910,0.471079,0.353086,0.076262
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000
max,63.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


(None, None, None)

## Encoding and balancing

In [34]:
# Define X and y
X = df.drop('y', axis=1)
y = df[target]

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=df[target], random_state=0)

## PyCaret

In [35]:
# Initialize PyCaret with the balanced data
clf = pycaret.setup(data=X, target=y,
            use_gpu=True, session_id=123,
            log_experiment = True, experiment_name = 'predict-term_deposit-marketing-2020',
            fix_imbalance_method="smote_tomek")

[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 3050 Laptop GPU, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 16 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Warning] GPU acceleration is disabled because no non-trivial dense features can be found
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leav

,Description,Value
0,Session id,123
1,Target,y
2,Target type,Binary
3,Original data shape,"(40000, 36)"
4,Transformed data shape,"(40000, 36)"
5,Transformed train set shape,"(28000, 36)"
6,Transformed test set shape,"(12000, 36)"
7,Numeric features,35
8,Preprocess,True
9,Imputation type,simple


[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 3050 Laptop GPU, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 16 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Warning] GPU acceleration is disabled because no non-trivial dense features can be found
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leav

Traceback (most recent call last):
  File "c:\Users\Guill\miniconda3\envs\term_deposit-post\Lib\site-packages\mlflow\store\tracking\file_store.py", line 327, in search_experiments
    exp = self._get_experiment(exp_id, view_type)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Guill\miniconda3\envs\term_deposit-post\Lib\site-packages\mlflow\store\tracking\file_store.py", line 421, in _get_experiment
    meta = FileStore._read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Guill\miniconda3\envs\term_deposit-post\Lib\site-packages\mlflow\store\tracking\file_store.py", line 1367, in _read_yaml
    return _read_helper(root, file_name, attempts_remaining=retries)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Guill\miniconda3\envs\term_deposit-post\Lib\site-packages\mlflow\store\tracking\file_store.py", line 1360, in _read_helper
    

[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 3050 Laptop GPU, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 16 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Warning] GPU acceleration is disabled because no non-trivial dense features can be found
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leav

In [36]:
def recall_class_1_function(y_true, y_pred, **kwargs):
    from sklearn.metrics import recall_score
    # Print or log y_pred to check if class 1 is being predicted
    print("y_pred distribution:", pd.Series(y_pred).value_counts())  # Logging prediction distribution
    return recall_score(y_true, y_pred, pos_label=1, **kwargs)

In [37]:
# Add the custom metric to PyCaret
pycaret.add_metric(
    id='recall_class_1',              # Unique identifier for the metric
    name='Recall Class 1',            # Display name
    score_func=recall_class_1_function, # The scoring function defined above
    greater_is_better=True,            # Recall should be maximized
    target='pred',                     # The target is the prediction, not probabilities
    multiclass=False                   # Not using multiclass, focus is on binary recall
)
best_model = pycaret.compare_models(
    sort='Recall Class 1',  # Sort models by the custom recall metric
    verbose=False           # Do not print out all the logs
)

Traceback (most recent call last):
  File "c:\Users\Guill\miniconda3\envs\term_deposit-post\Lib\site-packages\mlflow\store\tracking\file_store.py", line 327, in search_experiments
    exp = self._get_experiment(exp_id, view_type)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Guill\miniconda3\envs\term_deposit-post\Lib\site-packages\mlflow\store\tracking\file_store.py", line 421, in _get_experiment
    meta = FileStore._read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Guill\miniconda3\envs\term_deposit-post\Lib\site-packages\mlflow\store\tracking\file_store.py", line 1367, in _read_yaml
    return _read_helper(root, file_name, attempts_remaining=retries)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Guill\miniconda3\envs\term_deposit-post\Lib\site-packages\mlflow\store\tracking\file_store.py", line 1360, in _read_helper
    

In [38]:
tuned_model = pycaret.tune_model(best_model, optimize='recall_class_1',
                                 custom_scorer='recall_class_1',
                                #  threshold=0.3
                                 )

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,00:06:39
Status,. . . . . . . . . . . . . . . . . .,Searching Hyperparameters
Estimator,. . . . . . . . . . . . . . . . . .,Logistic Regression


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


TerminatedWorkerError: A worker process managed by the executor was unexpectedly terminated. This could be caused by a segmentation fault while calling the function or by an excessive memory usage causing the Operating System to kill the worker.


In [67]:
final_model = pycaret.finalize_model(tuned_model)

2024/09/19 09:23:39 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [68]:
# 2. Predict using the final model
predictions = pycaret.predict_model(final_model, data=X_test)

print(classification_report(y_test, predictions['prediction_label']))
print(roc_auc_score(y_test, predictions['prediction_label']))
print(td.recall_class_1_function(y_test, predictions['prediction_label']))

confusion_matrix(y_test, predictions['prediction_label'])

# Save the final model
pathlib.Path("../models/pyCaret/term_deposit_model").mkdir(
    parents=True, exist_ok=True
)
pycaret.save_model(final_model, '../models/pyCaret/term_deposit_model')

              precision    recall  f1-score   support

           0       0.97      0.44      0.61      7421
           1       0.10      0.81      0.18       579

    accuracy                           0.47      8000
   macro avg       0.53      0.63      0.39      8000
weighted avg       0.90      0.47      0.57      8000

0.6253967234373629
y_pred distribution: 1    4619
0    3381
Name: prediction_label, dtype: int64
0.8100172711571675
Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(exclude=None,
                                     include=['age', 'balance', 'day',
                                              'duration', 'campaign'],
                                     transformer=SimpleImputer(add_indicator=False,
                                                               copy=True,
                                                               fill_value=None,
                                                               keep_empty_features=False,
                                                               missing_values=nan,
                                                               strategy='mean'))),
                 ('categorical_imputer',
                  TransformerWrapper(exclude=None,
                                     include...
                                     include=['job', 'marital', 'education',
                 

## H2O

In [39]:
import h2o
from h2o.automl import H2OAutoML

# Start the H2O cluster (locally)
h2o.init()
conf_H2O = config['settings'].get('H2O')

# Convert your balanced train data to H2OFrame
df_train = pd.concat([X_train, y_train], axis=1)  # Combine features and target
df_test = pd.concat([X_test, y_test], axis=1)    # Combine features and target

# Define target variable
y = 'target'

# Convert your data
train = h2o.H2OFrame(df_train)
test = h2o.H2OFrame(df_test)

# Ensure response is a factor for binary classification
train[target] = train[target].asfactor()
test[target] = test[target].asfactor()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 22.0.2+9-70, mixed mode, sharing)
  Starting server from C:\Users\Guill\miniconda3\envs\term_deposit-post\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\Guill\AppData\Local\Temp\tmpo_ia4m5y
  JVM stdout: C:\Users\Guill\AppData\Local\Temp\tmpo_ia4m5y\h2o_Guill_started_from_python.out
  JVM stderr: C:\Users\Guill\AppData\Local\Temp\tmpo_ia4m5y\h2o_Guill_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,America/Mexico_City
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.5
H2O_cluster_version_age:,1 month and 7 days
H2O_cluster_name:,H2O_from_python_Guill_x5gsqo
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.839 Gb
H2O_cluster_total_cores:,0
H2O_cluster_allowed_cores:,0
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [95]:
import h2o
from h2o.automl import H2OAutoML
from sklearn.metrics import recall_score

# Initialize H2O
h2o.init()

# Define a custom recall function for class 1
class CustomRecallFunc:
    def map(self, pred, act, w, o, model):
        # Calculate recall for class 1
        true_positives = sum((act == 1) & (pred == 1))
        false_negatives = sum((act == 1) & (pred == 0))
        return [true_positives, true_positives + false_negatives]

    def reduce(self, l, r):
        return [l[0] + r[0], l[1] + r[1]]

    def metric(self, l):
        return l[0] / l[1] if l[1] > 0 else 0  # Recall = TP / (TP + FN)

# Upload the custom metric
custom_recall_func = h2o.upload_custom_metric(CustomRecallFunc, func_name="recall_class_1")

# Run AutoML for 20 base models with custom recall metric
aml = H2OAutoML(custom_metric_func=custom_recall_func, **conf_H2O)
aml.train(x=list(X_train.columns), y=target, training_frame=train)

# View the AutoML Leaderboard
lb = aml.leaderboard
lb = lb.sort("recall_class_1", ascending=False)  # Sort by custom recall metric
display(Markdown('## H2O Leaderboard'))
display(lb.head(rows=lb.nrows))

# Make predictions on the test set
predictions = aml.leader.predict(test)
display(Markdown('## Predictions'))
display(predictions.head())

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,19 secs
H2O_cluster_timezone:,America/Mexico_City
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.5
H2O_cluster_version_age:,1 month and 7 days
H2O_cluster_name:,H2O_from_python_Guill_x5gsqo
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.820 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"locked, healthy"


AutoML progress: |█
00:08:23.731: AutoML: XGBoost is not available; skipping it.

██████████████████████████████████████████████████████████████ (cancelled) 100%


In [97]:
mc_plot = aml.model_correlation_heatmap(
    frame=test
    )

KeyboardInterrupt: 

In [ ]:
# or if some subset of the models is needed a slice of leaderboard can be used, e.g., using MAE as the sorting metric
mc_plot = h2o.model_correlation_heatmap(aml.leaderboard.sort("mae").head(10), test)

In [ ]:
# or even extended leaderboard can be used
mc_plot = h2o.model_correlation_heatmap(h2o.automl.get_leaderboard(aml, extra_columns="training_time_ms").sort("training_time_ms").head(10), test)

In [ ]:
# also more complicated queries on leaderboard can be used, e.g., model correlation between 5 fastest models to train and Stacked Ensembles
leaderboard = h2o.automl.get_leaderboard(aml, extra_columns="training_time_ms").sort("training_time_ms")
mc_plot = h2o.model_correlation_heatmap(leaderboard.head(5).rbind(leaderboard[leaderboard["model_id"].grep("StackedEnsemble", output_logical=True)]), test)

In [213]:
aml.leader.fair_shap_plot(frame=test,
                          column='target',
                          protected_columns=["None"])

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


H2OResponseError: Server error java.lang.IllegalArgumentException:
  Error: Column None not found
  Request: POST /99/Rapids
    data: {'ast': "(tmp= py_18_sid_ab3e (unique (cols_py Key_Frame__upload_984acee654effc263cd4fb4f81a44444.hex 'None') False))", 'session_id': '_sid_ab3e'}
